## DESE61003 - Audio Experience Design
# Week 7 - Real-Time Audio Recognition

In [ ]:
try:
    import pythonosc
except ImportError:
    %pip install -q python-osc

from packaging.version import parse as parse_version
try:
    import disiple
    if parse_version(disiple.__version__) < parse_version('0.6.1'):
        raise ImportError('Outdated version')
except ImportError:
    %pip install -q --upgrade disiple

In [ ]:
from disiple.signals import AudioSignal, Spectrum, PowerSpectrum, Spectrogram, TimeSignal
import numpy as np
from dataclasses import dataclass
from collections import deque
from pythonosc.udp_client import SimpleUDPClient
import soundcard

In this final session, we'll tie everything we've learned so far together and build a real-time recognition system that takes live input from a microphone.

### 1. Block processing for audio analysis

The same principle of block-based processing that we used for (pseudo) real-time audio effects can also be applied to audio analysis, which we then call _streaming analysis_ (as opposed to _offline analysis_). Whereas audio effects have the strict constraint that each block needs to return as many output samples as the number of input samples it received, real-time analysis has much more freedom in its output. An analysis result can be returned for every sample or every block (e.g. a spectral centroid calculated from the surrounding N samples), but not every block needs to result in output. Output could also be generated only every X blocks or even irregularly (e.g. only when a certain event is recognised).

When the output is calculated in a regular interval, whether it is every sample or less often, the output of the analysis process will be a `TimeSignal`. Remember that we first encountered a `TimeSignal` in Session 5.1 as the more generic version of an `AudioSignal`. It is a one-dimensional time signal like the latter, but not constrained to have amplitudes between -1 and 1 (and consequently cannot be played back, but can be displayed using the `.display()` method). A good example is a signal representing the evolution of power over time. We did create such a `TimeSignal` in Activity 4 of Session 5.1, but we'll briefly revisit it here.

#### Activity 1

Create a function `power_over_time` that takes as input and `AudioSignal`, a frame size and a step size, and returns a `TimeSignal` containing the power for every frame constructed according to the given parameters. An outline of the function is given below. It needs to be modified such that the function call with `flute_signal` in the cell below it displays the power.

For reference, the implementation will consist of three parts. First a 2D NumPy array containing the frames of the input signal will need to be obtained. Then the power for each frame with length $N$ needs to be computed as $$P_N = \dfrac{1}{N}\sum_{n=0}^{N-1} x(n)$$ The result will be a 1D NumPy array. Finally, this array of powers needs to be passed to a `TimeSignal` constructor. You can also consult the solution of Session 5.1 for help.

In [ ]:
def power_over_time(input_signal, frame_size, step_size):
    # your code here
    return TimeSignal([], input_signal.samplerate)

In [ ]:
def power_over_time(input_signal, frame_size, step_size):
    frame_data = input_signal.frame_data(frame_size=frame_size, step_size=step_size)
    frame_power = np.mean(np.abs(frame_data)**2, axis=0)
    return TimeSignal(frame_power, input_signal.samplerate/step_size)

In [ ]:
flute_signal = AudioSignal('../data/Flute.nonvib.ff.A4.mono.wav')
frame_size = 512
step_size = 1
power_over_time(flute_signal, frame_size, step_size).display()

We will now create a block-based processing function that does the same. To start, that means we need a block-based equivalent of splitting a signal into frames. Just like for audio effects, we should not rely on the block size itself to have a particular value. The implementation of the delay effect has shown us a very relevant technique though. A circular buffer stores the most recent N samples, therefore it can be seen as containing an audio frame at any given time. Every time the circular buffer updates, the frame advances a sample, corresponding to a step size of 1, implementing a so-called _sliding window_.

To create a block-based equivalent of framing, we therefore use a `deque` again to contain the state of the block-processing function.

In [ ]:
@dataclass
class SlidingFrameState:
    circ_buffer: deque

The actual block-processing function that calculates the power per frame looks quite like the delay processing function. For this first version, let's stick to a step size of one, that will keep the implementation easier.

#### Activity 2

Implement a `sliding_frame_processor(input_buffer, state)` block processing function. Have a look at the solution to Activity 5 and 6 of Session 7.1 for reference. Very similarly, we need to iterate over the samples in each block and update the circular buffer with `state.circ_buffer.append(in_sample)`. We don't particularly need the first sample of the buffer, so the `.popleft()` method won't be used here. Instead, we want to use the buffer in its entirety. A `deque` can be converted into a NumPy array with `np.array(state.circ_buffer)` such that all of NumPy's numerical functionality can be used. Since we now have a NumPy array containing a frame, we can use the same code to calculate the as in Activity 1. The result should be stored in an output buffer.

In [ ]:
def sliding_frame_processor(input_buffer, state):
    output_buffer = []
    for in_sample in input_buffer:
        state.circ_buffer.append(in_sample)
        output_buffer.append(np.mean(np.array(state.circ_buffer) ** 2))
    return output_buffer

Because our block-processing function produces samples of a time signal that is not an audio signal, we can't use it in combination with the `AudioSignal.from_processing_blocks()` class method like we did when running audio effects. Luckily, the equivalent `TimeSignal.from_processing_blocks()` exists as well. Its usage is identical to the `AudioSignal` version. We first create a state object and initialise its only property, `circ_buffer` with a buffer of all zeros. Then we pass that state object together with block-processing function, block size and input signal to `TimeSignal.from_processing_blocks()`. Finally we call the `.display()` method of the resulting `TimeSignal`. If all went well, we can see a plot of power over time that is identical to the output produced in Activity 1.

In [ ]:
block_size = 256
state = SlidingFrameState(deque(np.zeros(frame_size), maxlen=frame_size))
sliding_power_signal = TimeSignal.from_processing_blocks(flute_signal, block_size, sliding_frame_processor, state)
sliding_power_signal.display()

Now that the sliding window version of our block-based framing equivalent is working, adding the option of a custom step size is not that much extra work. The main addition is some extra bookkeeping, for which we need to extend our state `dataclass`. We'll want to have the value of the step size available inside the block-processing function, so we will add that as an extra property. We also a `sample_counter` variable to keep track of how many samples we've seen in total.

In [ ]:
@dataclass
class FrameState:
    step_size: int
    sample_counter: int
    circ_buffer: deque

#### Activity 3

Create a `frame_processor(input_buffer, state)` processing function that extends the sliding with custom step size functionality. To do so you'll need increment the `sample_counter` property at every sample and only return the power of a frame when that `sample_counter` is a multiple of `step_size`. To test whether a number is an integer multiple of another, the modulo operator [`%`](https://numpy.org/doc/stable/reference/generated/numpy.remainder.html) can be used. For those steps that are not a multiple of `step_size`, the power does not need to be calculated at all, only the buffer needs updating. Therefore increasing the step size will decrease the computational requirements.

In [ ]:
def frame_processor(input_buffer, state):
    output_buffer = []
    for in_sample in input_buffer:
        state.circ_buffer.append(in_sample)
        state.sample_counter += 1
        if state.sample_counter % state.step_size == 0:
            output_buffer.append(np.mean(np.array(state.circ_buffer) ** 2))
    return output_buffer

The updated processing function can then be called as before. The state class is now expanded, and the step size and sample counter (initialised to 0) need to be set. Because a step size larger than 1 creates a new `TimeSignal` with a lower sample rate than the original `AudioSignal`, we also need to pass this lower rate to the `.from_processing_blocks()` method. The result can again be compared with the output of the `power_over_time` function created in Activity 1 and both should look the same.

In [ ]:
block_size = 256
step_size = 256
state = FrameState(step_size, 0, deque(np.zeros(frame_size), maxlen=frame_size))
power_signal = TimeSignal.from_processing_blocks(flute_signal, block_size, frame_processor, state, output_samplerate=flute_signal.samplerate/step_size)
power_signal.display()
power_over_time(flute_signal, frame_size, step_size).display()

### 2. Inter-process communication with OpenSoundControl (OSC)

When we are trying to recognise non-regular sound events, it does not make much sense to generate a `TimeSignal`. In theory, we could generate a signal that is zero by default and rises briefly to one to indicate the presence of a recognised event, but that is not the most intuitive. Instead it would be better if we could send an alert each time the event is detected, then we avoid creating data when nothing is detected. To send such alerts, we can use the [Open Sound Control](https://en.wikipedia.org/wiki/Open_Sound_Control) protocol. This makes it straightforward to communicate between different processes and programs (e.g. Max). Because it's a network protocol, the receiver can even be on another computer.

Using OSC with Python is straightforward. We first need to decide which IP address and port to send the messages to. Here we use the special value `localhost` to indicate that we want to send to our own computer.

In [ ]:
ip = 'localhost'
port = 6000

We then create a client object with these arguments.

In [ ]:
client = SimpleUDPClient(ip, port)

OSC messages can then be sent by calling the `.send_message()` method. They consist of two parts, the first being a string with the so-called OSC _address_ (not to be confused with the IP address). Typically these addresses follow a filesystem-like pattern, separated by forward slashes. The second argument is one or more values (or `None`) that will be sent to the address.

In [ ]:
client.send_message('/hello', None)
client.send_message('/with/value', 0)
client.send_message('/a/random/message', ['testing', 1, True, 3.])

We can verify that our messages are sent correctly by monitoring the port at the IP address we've chosen, for instance with the cross-platform [Protokol](https://hexler.net/protokol#get) software or with a Max patch.

To see this messaging capability in action, we're going to create a system that can recognise percussive events. We'll implement a streaming percussion detector by calculating the zero-crossing rate for each frame and if its value passes a threshold, the frame will be considered to contain a percussive event. Since this involves frame-wise processing, the `frame_processor` block-processing function of Activity 3 provides a good starting point.

First we need an implementation of the zero-crossing rate (ZCR), which is given below with an extra addition of a power threshold which we'll use to avoid recognising percussive events in low-power noise.

In [ ]:
def zero_crossing_rate(samples, power_threshold_dB):
    power = 10 * np.log10(np.mean(samples**2))
    zcr = np.count_nonzero(np.diff(np.sign(samples))) / len(samples)
    if power > power_threshold_dB:
        return zcr
    else:
        return zcr

We use a similar state `dataclass`, but extend it with parameters to store the threshold values the zero-crossing rate and power need to exceed in order to qualify as a percussive event. We also add the OSC client to pass around between subsequent calls to the block-processing function. In theory we could recreate the OSC client every time we want to send a message, by passing the IP address and port as a parameter instead, but that would create computational overhead. In general, it is best to keep the code of the block-processor as lightweight as possible, since it is repeated many times. Therefore create all variables that can be reused between blocks outside of the processing function and pass them around using the state object.

In [ ]:
@dataclass
class ZeroCrossingRateState:
    step_size: int
    sample_counter: int
    circ_buffer: deque
    zcr_threshold: float
    power_threshold_dB: float
    osc_client: SimpleUDPClient

#### Activity 4

Creat a `zero_crossing_rate_processor(input_buffer, state)` block-processing function that sends an OSC message whenever a given ZCR threshold is exceeded. The implementation will follow a general block-processing framing approach, meaning that you'll update the circular buffer with every sample and whenever the appropriate number of time steps have passed, the buffer is read in its entirety as `np.array(state.circ_buffer)`. The zero-crossing rate of this frame will then be calculated and if it exceeds the threshold parameter, an OSC message will be sent.

In [ ]:
def zero_crossing_rate_processor(input_buffer, state):
    output_buffer = []
    for in_sample in input_buffer:
        state.circ_buffer.append(in_sample)
        state.sample_counter += 1
        if state.sample_counter % state.step_size == 0:
            frame_samples = np.array(state.circ_buffer)
            zcr = zero_crossing_rate(frame_samples, state.power_threshold_dB)
            if zcr > state.zcr_threshold:
                state.osc_client.send_message('/zcr', zcr)
            output_buffer.append(zcr)
    return output_buffer

We will test the system with a signal that contains a mixture of humming and clapping. A successful percussion detector should ignore the humming and trigger on the clapping.

In [ ]:
humming_clapping_signal = AudioSignal('../data/humming-clapping.wav')
humming_clapping_signal.play()

If your implementation is correct, the code snippet below should run the block-processing function frame by frame over the given audio signal. It is not strictly necessary to write to an output buffer in `zero_crossing_rate_processor`, in which case the result of `TimeSignal.from_processing_blocks()` will be `None`. However, in order to experiment with and correctly set the various parameters, it could be useful to write the ZCR values into an output buffer. Then you can plot the resulting `TimeSignal`, and visually assess the parameter settings.

In [ ]:
block_size = 256
frame_size = 4096
step_size = 4096
state = ZeroCrossingRateState(step_size, 0, deque(np.zeros(frame_size), maxlen=frame_size), 0.15, -40, SimpleUDPClient(ip, port))
zcr_signal = TimeSignal.from_processing_blocks(humming_clapping_signal, block_size, zero_crossing_rate_processor, state, output_samplerate=humming_clapping_signal.samplerate/step_size)
if zcr_signal is not None:
    zcr_signal.display()

### 3. Using live audio input

We got as far as it goes with simulating real-time processing, but there's nothing like the real deal. So now we'll see how we can get a live audio input into our notebook. We'll be using the [SoundCard](https://soundcard.readthedocs.io) library for this, which has already been imported as `soundcard` at the top of this notebook.

Since we want to get audio input, we start be checking which microphones (or generally audio input devices) are available on this computer.

In [ ]:
soundcard.all_microphones()

If you see a specific device you want to use, you can select is by passing its (partial) name to the [`get_microphone()`](https://soundcard.readthedocs.io/en/latest/#soundcard.get_microphone) function.

In [ ]:
mic = soundcard.get_microphone('Built-in')
mic

Or we can just go with the default selection.

In [ ]:
mic = soundcard.default_microphone()
mic

Conceptually, there is nothing different between processing an audio file or a live input in blocks, so it shouldn't come as a surprise that the `.from_processing_block()` class method has a direct counterpart `.from_processing_recording()`. Instead of passing an `AudioSignal` to process as the first argument, you pass two arguments: a `soundcard` mic object and a recording samplerate. The rest of the arguments is the same, so the complete type signature is `.from_processing_recording(recording_device, recording_samplerate, block_size, processing_fn, state_data=None, output_samplerate=None)`. The last two arguments are optional, defaulting to no state and `output_samplerate` equalling `recording_samplerate` respectively. Using `.from_processing_recording()` with either a `TimeSignal` or an `AudioSignal`, we can reuse the same block-processing functions for audio effects and streaming analysis that we created for block-based file processing earlier.

The easiest audio effect is still one that does nothing, but since we're now recording from live input and storing the result, a passthrough effect is no longer useless. Combined with `.from_processing_recording()` it will implement an audio recorder. To start recording, you simply execute the cell. The recording will run for an indefinite time, so you'll need to stop it manually by clicking the ⬛️ button in the toolbar or by selecting the menu option "Kernel ▷ Interrupt Kernel".

#### Activity 5

Create a passthrough block-processing function (or peek at Session 7.1) and pass it to `AudioSignal.from_processing_recording()`. Experiment with different block sizes and play back the recorded signal.

In [ ]:
def passthrough(input_buffer, _):
    return input_buffer

block_size = 1024
samplerate = 44100
recorded_signal = AudioSignal.from_processing_recording(mic, samplerate, block_size, passthrough)
recorded_signal.play()
recorded_signal.display()

Now that we know how to apply an audio effect to live input, we can do the same for streaming analysis of live input.

#### Activity 6

Use the `zero_crossing_rate_processor` block-processing function to detect percussive events in live input.

In [ ]:
block_size = 128
state = ZeroCrossingRateState(step_size, 0, deque(np.zeros(frame_size), maxlen=frame_size), 0.15, -40, SimpleUDPClient(ip, port))
recorded_zcr_signal = TimeSignal.from_processing_recording(mic, samplerate, block_size, zero_crossing_rate_processor, state, output_samplerate=samplerate/step_size)
if recorded_zcr_signal is not None:
    recorded_zcr_signal.display()

Recording without having any auditive feedback does make it harder to diagnose the problem when someone does not go as expected. It is impossible to know whether the problem lies with the recording itself or with the processing. Therefore it would be convenient if we could monitor our recordings. Luckily `soundcard` also includes functionality for live playback.

We can query what speakers we have available (do you headphones to avoid microphone feedback though).

In [ ]:
soundcard.all_speakers()

We can again select a particular device by name.

In [ ]:
speaker = soundcard.get_speaker('Built-in')
speaker

Or go with the default.

In [ ]:
speaker = soundcard.default_speaker()
speaker

To add a live monitor, we can once more use the block-processing framework. Let's create a recorder with monitor to demonstrate. We start by declaring a state class that stores the monitor device.

In [ ]:
@dataclass
class MonitorState:
    monitor: object

The syntax to play back a buffer of samples with `soundcard` is straightforward: [`.play(samples)`](https://soundcard.readthedocs.io/en/latest/#soundcard._Player.play), so adding it to our passthrough block-processor is easy.

In [ ]:
def monitor_passthrough(input_buffer, state):
    state.monitor.play(input_buffer)
    return input_buffer

This new block-processing function can then be passed to `AudioSignal.from_processing_recording()`, but some extra setup is needed to start the monitor device and shut it down gracefully up after usage.

In [ ]:
block_size = 1024
samplerate = 24000
with speaker.player(samplerate=samplerate) as monitor:
    state = MonitorState(monitor)
    recorded_signal = AudioSignal.from_processing_recording(mic, samplerate, block_size, monitor_passthrough, state)
recorded_signal.play()
recorded_signal.display()

Note that adding a monitor is quite resource-intensive, so the act of adding a monitor might itself disturb the recording process and cause crackling. Do increase the block size if needed and experiment with different sample rates. Certain sample rates might not be natively supported by your device and therefore need on-the-fly resampling, which increases the chance of dropped samples. This could mean that in certain situations, increasing the sample rate (from a non-supported to a natively supported one) might paradoxically decrease resource usage, and therefore allow a smaller block size.

### 4. Going further

Now that you know how to obtain live input from a microphone, you can create live versions of all effects and recognition algorithms we've seen so far. You can also add live monitoring, which in the case of audio effects can either monitor the input or the processed result. In the later case, the live playback could entirely replace the generation of a processed signal.